In [1]:
from util import connect_to_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData
from pandas import DataFrame

In [2]:
eng = connect_to_database()

In [53]:
# show table names in the database
eng.table_names()

['across_canonical',
 'across_pnl',
 'across_pools',
 'chains',
 'prices',
 'relayer_balance',
 'tokens',
 'transfers']

#### Q1 Find the max transfer size of each asset type. What was its USD amount?

In [54]:
query_q1 = """
with max_token as (select strftime('%Y-%m-%d',dt) as dt,
src_symbol,
max(src_amount) as amount
from transfers
group by src_symbol)

select a.dt,
a.src_symbol,
a.amount,
a.amount*b.price as usd_amount
from max_token a
left join prices b
on a.dt = b.dt and a.src_symbol = b.symbol
"""

In [55]:
result_q1 = eng.execute(query_q1)

In [56]:
df_q1 = DataFrame(result_q1.fetchall(), columns = result_q1.keys()).round(2)

In [57]:
df_q1

,dt,src_symbol,amount,usd_amount
0,2022-01-15,BADGER,7102.17,88983.87
1,2021-12-04,DAI,300000.00,300000.00
2,2022-02-11,ETH,1350.00,4160580.53
3,2021-12-04,MATIC,420000.00,959753.41
4,2022-01-30,UMA,122.84,746.07
5,2022-02-28,USDC,2949154.00,2949154.00
6,2021-11-08,USDT,307820.00,307820.00
7,2022-02-10,WBTC,10.00,444362.31


#### Q2 Find the day that each address that has used Across sent their first transaction.

In [61]:
query_q2 = """
select 
dst_address,
min(dt) over(partition by dst_address) as dt,
transaction_id,
src_symbol,
src_amount
from transfers
where bridge = 'across'
"""

In [62]:
result_q2 = eng.execute(query_q2)

In [63]:
df_q2 = DataFrame(result_q2.fetchall(), columns = result_q2.keys())
df_q2

,dst_address,dt,transaction_id,src_symbol,src_amount
0,0x00000000Cc0B822819f03424DAcf9077FdAa58A3,2021-11-17 09:46:34.000000,b539b4f0ce7a95f6ca6a00718e393b7e35c4fa9a990353...,ETH,0.100000
1,0x0000000813B34008A225De08a6a61835508C71f9,2021-12-18 09:16:18.000000,7b968bcc40158b35fdba4bf299ded84cee0d2a21c48db8...,ETH,0.601000
2,0x00050C4A4f685F1045AdccBc63223b8983a2C01c,2021-11-17 10:58:30.000000,aa200549bf4e0738f38d7452c088c062cb2683441226ca...,ETH,0.166000
3,0x00063dDB30be7Bc2292583D5f143E9d6E6228440,2022-02-02 03:20:58.000000,ad2fb881802250f4900681af1af953358c77baa2276b59...,ETH,1.001246
4,0x000CD92d64c8199b00f42Cb0218EBa60D09BEC60,2022-02-19 11:30:46.000000,710903ca936f951e4dbf170b86f56e4cbfd4c266c5308a...,ETH,1.946000
...,...,...,...,...,...
8992,0xffBFaD323308e04aA1c9DdC48a2f84121Ad223d4,2021-12-10 09:25:34.000000,be6c1e01ca2d70961c6e25e02d23f22988c3af74d27c35...,ETH,0.300000
8993,0xffFEc73Deb35b3ed9752e9dd33dAc5A20a341C01,2021-12-12 08:40:07.000000,6d74e4a27e767611d7ddd68cae50be5c0b84a977c459c3...,ETH,0.105000
8994,0xffd0B16Ad371A90676c4442b4065EA01Cf500E11,2022-01-27 07:38:03.000000,91e226002789c191613f7efc4d28675240db9476e7b3d1...,ETH,0.220000
8995,0xfffBDa353405BddC2a35f623307E5b5D78c60B8d,2021-12-13 13:48:25.000000,84579cfea0b0b6a3874abac48ff6c586dc5ebd9244ac7f...,ETH,0.221099


#### Q3 What's the maximum usd amount sent by each address that has used Across?

In [78]:
query_q3 = """
with tx as (select strftime('%Y-%m-%d',dt) as dt,
dst_address,
src_symbol,
src_amount
from transfers
where bridge = 'across'),

tx_us as (select a.dt,
a.dst_address,
a.src_symbol,
a.src_amount,
a.src_amount*b.price as usd_amount
from tx a
left join prices b
on a.dt = b.dt and a.src_symbol = b.symbol)

select dt,
dst_address,
max(usd_amount) as max_amount
from tx_us
group by dst_address
order by max(usd_amount) desc
"""

In [79]:
result_q3 = eng.execute(query_q3)

In [80]:
df_q3 = DataFrame(result_q3.fetchall(), columns = result_q3.keys()).round(2)
df_q3

,dt,dst_address,max_amount
0,2021-12-03,0xE9EE83812FA9eff257ab4f5F4bBd222a1D32f191,4519441.03
1,2022-02-11,0x9c5083dd4838E120Dbeac44C052179692Aa5dAC5,4160580.53
2,2022-03-19,0x8528651793c65142255f137249b5dB80714a6Cf6,3828272.55
3,2022-02-28,0x4DAEf97203Fa6654F6338872DA5bD1EA2E879533,2949154.00
4,2022-03-19,0xEDcFe14e38EF230747F7204324499Dd7DaE063dC,2702226.07
...,...,...,...
6349,2021-12-11,0x7B413AAB9E10a3e680E62c2f24328108eB32aE94,39.18
6350,2021-12-24,0xd8BA30E1D96E201767FeE9F7C43399c89810C5Bf,38.26
6351,2021-12-14,0x38AE9F3f915df560EB9Bf8E48FAcfe91F50D36D4,37.83
6352,2021-12-14,0x7DE4Cb13bbc634FAa432A65fE812048699442CA7,37.83


#### Q4 Find the 5 addresses that have generated the most bridge volume. What percent of Across's total volume was sent by those addresses?

In [49]:
query_q4 = """
with tx as (select strftime('%Y-%m-%d',dt) as dt,
dst_address,
src_symbol,
src_amount
from transfers
where bridge = 'across'),

tx_us as (select a.dt,
a.dst_address,
a.src_symbol,
a.src_amount,
a.src_amount*b.price as usd_amount
from tx a
left join prices b
on a.dt = b.dt and a.src_symbol = b.symbol)

select dst_address,
sum(usd_amount) as total_volume,
sum(usd_amount)/(select sum(usd_amount) from tx_us) as percentage
from tx_us
group by dst_address
order by sum(usd_amount) desc
limit 5
"""

In [50]:
result_q4 = eng.execute(query_q4)

In [51]:
df_q4 = DataFrame(result_q4.fetchall(), columns = result_q4.keys()).round(2)

In [52]:
df_q4

,dst_address,total_volume,percentage
0,0xE9EE83812FA9eff257ab4f5F4bBd222a1D32f191,15185643.22,0.09
1,0x6F58ead26d17DE1a9AadbD0309D9cfF2911930c2,8489251.38,0.05
2,0xfF33f5653E547A0b54b86b35A45e8b1C9aBD1C46,7432687.44,0.04
3,0x9c5083dd4838E120Dbeac44C052179692Aa5dAC5,4160580.53,0.02
4,0x98273FBdE2fC9695F63e7da7bAa489dC84719725,4007512.45,0.02
